In [1]:
import requests
import json
import pandas as pd
import csv
import psycopg2

In [2]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [3]:
from dotenv import load_dotenv
import os

In [4]:
def configure():
    load_dotenv()

# Extraction Layer

In [5]:
configure()
url = os.getenv('url')

querystring = {"limit":"100000"}

headers = {
	"x-rapidapi-key": os.getenv('x-rapidapi-key'),
	"x-rapidapi-host": os.getenv('x-rapidapi-host')
}

response = requests.get(url, headers=headers, params=querystring)

#print(response.json())

data = response.json()

# Save the data to a file
filename = "PropertyRecords.json"
with open(filename, "w") as file:
    json.dump(data, file, indent=4)

In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [7]:
# read into a dataframe

ProjectRecords_pd = pd.read_json('PropertyRecords.json')

In [8]:
ProjectRecords_pd.head()

,addressLine1,city,state,zipCode,formattedAddress,addressLine2,county,features,id,longitude,latitude,propertyType,lotSize,lastSalePrice,lastSaleDate,assessorID,bedrooms,ownerOccupied,squareFootage,yearBuilt,bathrooms,taxAssessment,propertyTaxes,owner,legalDescription,subdivision,zoning
0,1741 Nw 9th St,Oklahoma City,OK,73106,"1741 Nw 9th St, Apt 2, Oklahoma City, OK 73106",Apt 2,Oklahoma,{},"1741-Nw-9th-St,-Apt-2,-Oklahoma-City,-OK-73106",-97.540842,35.478013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Country Shire Ln,Spring Grove,IL,60081,"Country Shire Ln, Lot 205, Spring Grove, IL 60081",Lot 205,McHenry,NaN,"Country-Shire-Ln,-Lot-205,-Spring-Grove,-IL-60081",-88.205030,42.447925,Land,40946.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,205 N Main St,Pocahontas,IA,50574,"205 N Main St, Pocahontas, IA 50574",NaN,Pocahontas,{},"205-N-Main-St,-Pocahontas,-IA-50574",-94.669316,42.734575,NaN,NaN,45000.0,2023-05-17T00:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,287 Arbor Glen Blvd,Schaumburg,IL,60195,"287 Arbor Glen Blvd, Schaumburg, IL 60195",NaN,Cook,"{'cooling': True, 'coolingType': 'Central', 'exteriorType': 'Masonry', 'fireplace': True, 'floorCount': 2, 'foundationType': 'Slab', 'garage': True, 'garageSpaces': 2, 'garageType': 'Attached Garage', 'roofType': 'Shingle (Not Wood)', 'unitCount': 1}","287-Arbor-Glen-Blvd,-Schaumburg,-IL-60195",-88.087408,42.063833,Condo,4334.0,420000.0,2019-12-13T00:00:00.000Z,07-03-100-032,4.0,1.0,3056.0,2004.0,3.0,"{'2018': {'value': 38500, 'land': 2058, 'improvements': 36442}}",{'2018': {'total': 9699}},"{'names': ['JOHNNY KANG'], 'mailingAddress': {'id': '287-Arbor-Glen-Blvd,-Schaumburg,-IL-60195', 'addressLine1': '287 Arbor Glen Blvd', 'city': 'Schaumburg', 'state': 'IL', 'zipCode': '60195'}}",NaN,NaN,NaN
4,8916 W Rutter Pkwy,Spokane,WA,99208,"8916 W Rutter Pkwy, Spokane, WA 99208",NaN,Spokane,"{'architectureType': 'Ranch', 'cooling': True, 'coolingType': 'Central', 'exteriorType': 'Wood Frame', 'fireplace': True, 'floorCount': 1, 'foundationType': 'OTHER', 'garage': True, 'garageType': 'Mixed', 'heating': True, 'heatingType': 'Forced Air', 'roofType': 'Composition Shingle', 'roomCount': 4, 'unitCount': 1}","8916-W-Rutter-Pkwy,-Spokane,-WA-99208",-117.535895,47.775943,Single Family,230868.0,1000.0,2003-10-30T00:00:00.000Z,26053.9033,2.0,1.0,1663.0,1979.0,2.5,"{'2022': {'value': 421950, 'land': 145650, 'improvements': 276300}}",{'2023': {'total': 4974}},"{'names': ['STEVEN C GLORE'], 'mailingAddress': {'id': '8916-W-Rutter-Pkwy,-Spokane,-WA-99208', 'addressLine1': '8916 W Rutter Pkwy', 'city': 'Spokane', 'state': 'WA', 'zipCode': '99208'}}",05-26-42 PTN OF SW1/4 DAF; BEG 1837.74FT W OF S1/4 COR OF S EC & TRUE POB TH CONT W65FT TH N1705.99FT TH N62DEG 47MIN 02SDS E,NaN,NaN


In [9]:
ProjectRecords_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   addressLine1      500 non-null    object 
 1   city              500 non-null    object 
 2   state             500 non-null    object 
 3   zipCode           500 non-null    int64  
 4   formattedAddress  500 non-null    object 
 5   addressLine2      94 non-null     object 
 6   county            500 non-null    object 
 7   features          457 non-null    object 
 8   id                500 non-null    object 
 9   longitude         500 non-null    float64
 10  latitude          500 non-null    float64
 11  propertyType      410 non-null    object 
 12  lotSize           355 non-null    float64
 13  lastSalePrice     255 non-null    float64
 14  lastSaleDate      322 non-null    object 
 15  assessorID        360 non-null    object 
 16  bedrooms          329 non-null    float64
 1

In [10]:
#location
#description
#

In [11]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [12]:
ProjectRecords_pd['squareFootage'].nunique()


334

# Transformation Layer

In [13]:
#Coverting dictionary columns into string
def convert_dict_columns_to_string(ProjectRecords_pd: pd.DataFrame) -> pd.DataFrame:
    """
    Converts all dictionary-type columns in a DataFrame to string format.

    Args:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: A DataFrame with dictionary columns converted to strings.
    """
    for col in ProjectRecords_pd.columns:
        # Check if the column contains dictionary entries
        if ProjectRecords_pd[col].apply(lambda x: isinstance(x, dict)).any():
            ProjectRecords_pd[col] = ProjectRecords_pd[col].apply(lambda x: str(x) if isinstance(x, dict) else x)
    return ProjectRecords_pd
ProjectRecords_pd = convert_dict_columns_to_string(ProjectRecords_pd)

In [14]:
# Replace Nan Values with appropraite defaults or remove row/columns as neccesary
ProjectRecords_pd.fillna({
    "assessorID": "Unknown",
    "legalDescription": "Not available",
    "squareFootage": 0,
    "subdivision": "Not available",
    "yearBuilt": 0,
    "bathrooms": 0,
    "lotSize": 0,
    "propertyType": "Unknown",
    "lastSalePrice":0,
    "lastSaleDate": "Not available",
    "features": "None",
    "taxAssessment": "Not available",
    "owner": "Unknown",
    "propertyTaxes": "Not available",
    "bedrooms": 0,
    "ownerOccupied": 0,
    "zoning": "Unknown",
    "addressLine2": "Not available",
    "formattedAddress": "Not Available",
    "county": "Not available"
    
}, inplace=True)

In [15]:
# Converting year built colummn from float to Integer
ProjectRecords_pd['yearBuilt'] = ProjectRecords_pd['yearBuilt'].astype(int)
ProjectRecords_pd['lastSaleDate'] = pd.to_datetime(ProjectRecords_pd['lastSaleDate'], errors='coerce')

In [16]:
# Create location Dimension
location_dim = ProjectRecords_pd[["city", "county", "state"]].drop_duplicates()
location_dim = location_dim.reset_index(drop=True).reset_index().rename(columns={"index": "location_id"})
location_dim["location_id"] += 1  # Start index from 1 instead of 0
location_dim.head(10)

,location_id,city,county,state
0,1,Oklahoma City,Oklahoma,OK
1,2,Spring Grove,McHenry,IL
2,3,Pocahontas,Pocahontas,IA
3,4,Schaumburg,Cook,IL
4,5,Spokane,Spokane,WA
5,6,Hitchcock,Galveston,TX
6,7,Bronx,Bronx,NY
7,8,Seneca,Grant,OR
8,9,Lincoln,Lancaster,NE
9,10,Lubbock,Lubbock,TX


In [17]:
# Create Property Features Dimension
description_dim = ProjectRecords_pd[["propertyType", "bathrooms", "bedrooms", "yearBuilt", "squareFootage", "lotSize"]].drop_duplicates().reset_index(drop=True)
description_dim  = description_dim.reset_index().rename(columns={"index": "description_id"})
description_dim ["description_id"] += 1  # Start index from 1 instead of 0


In [18]:
description_dim.head(30)

,description_id,propertyType,bathrooms,bedrooms,yearBuilt,squareFootage,lotSize
0,1,Unknown,0.0,0.0,0,0.0,0.0
1,2,Land,0.0,0.0,0,0.0,40946.0
2,3,Condo,3.0,4.0,2004,3056.0,4334.0
3,4,Single Family,2.5,2.0,1979,1663.0,230868.0
4,5,Duplex-Triplex,2.0,6.0,1930,3524.0,3005.0
5,6,Manufactured,0.0,0.0,1979,0.0,7405.0
6,7,Single Family,2.0,4.0,1910,1792.0,6970.0
7,8,Single Family,2.0,3.0,2008,1603.0,5656.0
8,9,Single Family,4.5,4.0,2013,3415.0,46967.0
9,10,Single Family,2.0,3.0,1986,1830.0,8712.0


In [19]:
# Create time Dimension
time_dim = ProjectRecords_pd[["lastSaleDate"]].drop_duplicates().reset_index(drop=True)
time_dim = time_dim.reset_index().rename(columns={"index": "time_id"})
time_dim["time_id"] += 1  # Start index from 1 instead of 0
time_dim['year'] = time_dim['lastSaleDate'].dt.isocalendar().year  # year
time_dim['week'] = time_dim['lastSaleDate'].dt.isocalendar().week  # ISO week number
time_dim['month'] = time_dim['lastSaleDate'].dt.month_name()            # Month number
time_dim['quarter'] = time_dim['lastSaleDate'].dt.quarter          # Quarter (1-4)
time_dim['day_of_week'] = time_dim['lastSaleDate'].dt.day_name()   # Full weekday name


In [20]:
time_dim.head()

,time_id,lastSaleDate,year,week,month,quarter,day_of_week
0,1,NaT,<NA>,<NA>,NaN,NaN,NaN
1,2,2023-05-17 00:00:00+00:00,2023,20,May,2.0,Wednesday
2,3,2019-12-13 00:00:00+00:00,2019,50,December,4.0,Friday
3,4,2003-10-30 00:00:00+00:00,2003,44,October,4.0,Thursday
4,5,2024-04-17 00:00:00+00:00,2024,16,April,2.0,Wednesday


In [21]:
# Create the fact table and linking of dimension table primary key with fact table
fact_columns = ["addressLine1", "city", "lastSaleDate", "propertyType", "bathrooms", "bedrooms", "yearBuilt", "squareFootage", "lotSize", 'lastSalePrice']
fact_table = ProjectRecords_pd[fact_columns]
fact_table = fact_table.merge(time_dim[["lastSaleDate", "time_id"]], on="lastSaleDate", how="left")
fact_table = fact_table.merge(location_dim[["city", "location_id"]], on="city", how="left")
fact_table = fact_table.merge(description_dim, on=["propertyType", "bathrooms", "bedrooms", "yearBuilt", "squareFootage", "lotSize"], how="left")

fact_table .drop(columns=["lastSaleDate","city","propertyType", "bathrooms", "bedrooms", "yearBuilt", "squareFootage", "lotSize"], inplace=True)


In [22]:
fact_table.tail(40)

,addressLine1,lastSalePrice,time_id,location_id,description_id
496,3466 Deer Creek Palladian Cir,310000.0,293,383,382
497,2430 Evalon St,0.0,294,384,383
498,347 S 300 E,0.0,1,385,384
499,7825 Cohn St,234000.0,295,220,385
500,7825 Cohn St,234000.0,295,248,385
501,187 Walnut Dr,40000.0,198,386,386
502,2432 Tattersall Dr,120000.0,296,387,82
503,18 Silk St,0.0,1,388,387
504,1704 Marlyn Dr,108000.0,297,346,388
505,5751 W Irving Park Rd,0.0,1,23,1


In [23]:
# Saving fact and dimensions table in csv format
fact_table.to_csv("property_fact.csv", index=False)
location_dim.to_csv("location_dimension.csv", index=False)
description_dim.to_csv("description_dimension.csv", index=False)
time_dim.to_csv("time_dimension.csv", index=False)

# Loading Layer

In [24]:
# develop a function to connect to pgadmin

def get_db_connection():
    load_dotenv()
    connection = psycopg2.connect(
        host=os.getenv('postgres_host'),
        database=os.getenv('postgres_database'),
        user=os.getenv('postgres_user'),
        password=os.getenv('postgres_password')
    )
    return connection

conn = get_db_connection()



In [36]:
time_dim['year'].min()

1978

In [26]:
# Creating tables
def create_tables():
    conn = get_db_connection()
    cursor = conn.cursor()
    create_table_query =  '''CREATE SCHEMA IF NOT EXISTS zapbank;
    
                             DROP TABLE IF EXISTS zapbank.fact_table;
                             DROP TABLE IF EXISTS zapbank.location_dim;
                             DROP TABLE IF EXISTS zapbank.sales_dim;
                             DROP TABLE IF EXISTS zapbank.dates_dim;
                             DROP TABLE IF EXISTS zapbank.description_dim;
                             
                             CREATE TABLE zapbank.fact_table(
                                 addressLine1 VARCHAR(255),
                                 lastSalePrice FLOAT,
                                 time_id INTEGER,
                                 location_id FLOAT,
                                 description_id FLOAT
                             );
                             
                             CREATE TABLE zapbank.location_dim(
                                 location_id SERIAL PRIMARY KEY,
                                 city VARCHAR(100),
                                 county VARCHAR(50),
                                 state VARCHAR(50)
                             );
                             
                             CREATE TABLE zapbank.description_dim(
                                 description_id SERIAL PRIMARY KEY,
                                 propertyType VARCHAR(50),
                                 bathrooms FLOAT,
                                 bedrooms FLOAT,
                                 yearBuilt INTEGER, 
                                 squareFootage FLOAT,
                                 lotSize FLOAT
                             ); 
                             
                             CREATE TABLE zapbank.dates_dim(
                                 time_id SERIAL PRIMARY KEY,
                                 lastSaleDate DATE,
                                 year INTEGER,
                                 week INTEGER,
                                 month VARCHAR(50),
                                 quater FLOAT,
                                 day_of_week VARCHAR(50)   
                             );'''
    cursor.execute(create_table_query)
    conn.commit()
    cursor.close()
    conn.close()
    
create_tables()

In [27]:
# create a function to load the csv data into the database
def load_data_from_csv_to_table(csv_path,  table_name):
    conn = get_db_connection()
    cursor = conn.cursor()
    with open(csv_path, 'r', encoding="utf-8") as file:
            reader = csv.reader(file)
            next(reader)
            for row in reader:
                placeholders = ', '.join(['%s'] * len(row))
                query = f'INSERT INTO {table_name} VALUES({placeholders});'
                cursor.execute(query, row)
    conn.commit()
    cursor.close()
    conn.close()     

In [28]:
# fact table
fact_csv_path = r'/Users/OYETAYOADEDAMOLA/Documents/AmdariProject/Project1/property_fact.csv'
load_data_from_csv_to_table(fact_csv_path,'zapbank.fact_table')

In [29]:
# location dimension table
location_csv_path = r'/Users/OYETAYOADEDAMOLA/Documents/AmdariProject/Project1/location_dimension.csv'
load_data_from_csv_to_table(location_csv_path,'zapbank.location_dim')

In [30]:
# description dimension table
description_csv_path = r'/Users/OYETAYOADEDAMOLA/Documents/AmdariProject/Project1/description_dimension.csv'
load_data_from_csv_to_table(description_csv_path,'zapbank.description_dim')

In [31]:
# create a function to load the csv data into the database (specifically for sales)
def load_data_from_csv_to_sales_table(csv_path, table_name):
    conn = get_db_connection()
    cursor = conn.cursor()
    with open(csv_path, 'r', encoding="utf-8") as file:
            reader = csv.reader(file)
            next(reader)
            for row in reader:
                # convert empty strings (or 'Not available') in date column to None(Null in SQL)
                row = [None if (cell == "" or cell == "Not available") else cell for cell in row]
                placeholders = ', '.join(['%s'] * len(row))
                query = f'INSERT INTO {table_name} VALUES({placeholders});'
                cursor.execute(query, row)
    conn.commit()
    cursor.close()
    conn.close()


In [32]:
# time dimension table
time_csv_path = r'/Users/OYETAYOADEDAMOLA/Documents/AmdariProject/Project1/time_dimension.csv'
load_data_from_csv_to_sales_table(time_csv_path,'zapbank.dates_dim')

In [33]:
print('All Data has been loaded successfully into their respective schema and tables')

All Data has been loaded successfully into their respective schema and tables
